In [ ]:
import io
import sys
import numpy as np
import math

# colours
RED = '\033[91m'
GREEN = '\033[92m'
RESET = '\033[0m'  # Reset color

In [2]:
def test_stable_calculation():
    def run_individual_test(test_number, description, x_value, expected_output):
        try:
            result = stable_calculation(x_value)
            # Check if the function is implemented
            if result is None:
                raise NotImplementedError("stable_calculation function is not implemented")

            assert np.isclose(result, expected_output), f"Expected {expected_output}, got {result}"
            print(f"{GREEN}Test {test_number} Passed: {description}{RESET}")

        except NotImplementedError as e:
            print(f"{RED}Test {test_number} Skipped: {description}. {e}{RESET}")
        except AssertionError as e:
            print(f"{RED}Test {test_number} Failed: {description}. {str(e)}{RESET}")

    # Running the tests with explicit values and expected outputs
    run_individual_test(1, "Stable calculation with small x value", 1, 1 / (np.sqrt(2) + np.sqrt(1)))
    run_individual_test(2, "Stable calculation with large x value", 1e16, 1 / (np.sqrt(1e16 + 1) + np.sqrt(1e16)))

In [ ]:

def test_numpy_challenge():

    # -----------------------------
    # create_array(x)
    # -----------------------------
    def test_create_array():
        def run_individual_test(test_number, description, x_value):
            try:
                result = create_array(x_value)
                if result is None:
                    raise NotImplementedError("create_array function is not implemented")
                assert isinstance(result, np.ndarray), "Output is not a NumPy array"
                assert result.shape == (x_value, x_value), f"Array shape is not ({x_value}, {x_value})"
                assert result.min() >= 1 and result.max() <= 100, "Array values not in expected range (1-100)"
                assert np.issubdtype(result.dtype, np.integer), "Array dtype should be integer"
                print(f"{GREEN}Test {test_number} Passed: {description}{RESET}")
            except (NotImplementedError, AssertionError, ValueError, TypeError) as e:
                print(f"{RED}Test {test_number} Failed/Skipped: {description}. {str(e)}{RESET}")

        # Happy paths
        run_individual_test(1, "Create 5x5 array", 5)
        run_individual_test(2, "Create 1x1 array", 1)


    # -----------------------------
    # slice_array(arr, slice_size)
    # -----------------------------
    def test_slice_array():
        def run_individual_test(test_number, description, arr, slice_size, expected_output):
            try:
                result = slice_array(arr, slice_size)
                if expected_output is None:
                    assert result is None, f"Expected None, got {result}"
                else:
                    if result is None:
                        raise AssertionError("slice_array returned None but a valid slice was expected")
                    assert isinstance(result, np.ndarray), "slice_array should return a NumPy array"
                    assert result.shape == expected_output.shape, f"Sliced array shape is not {expected_output.shape}"
                    assert np.array_equal(result, expected_output), "Sliced array does not match expected central region"
                print(f"{GREEN}Test {test_number} Passed: {description}{RESET}")
            except (NotImplementedError, AssertionError, ValueError, TypeError) as e:
                print(f"{RED}Test {test_number} Failed/Skipped: {description}. {str(e)}{RESET}")

        # Base happy paths already present
        arr = np.arange(25).reshape(5, 5)
        expected_3x3 = arr[1:4, 1:4]
        run_individual_test(3, "Central 3x3 slice from 5x5 array (odd/odd)", arr, 3, expected_3x3)

        arr2 = np.arange(16).reshape(4, 4)
        expected_2x2 = arr2[1:3, 1:3]
        run_individual_test(4, "Central 2x2 slice from 4x4 array (even/even)", arr2, 2, expected_2x2)

        # Parity mismatch: odd array, even slice -> None
        run_individual_test(7, "Odd array (5x5), even slice_size=2 -> None", arr, 2, None)

        # Parity mismatch: even array, odd slice -> None
        run_individual_test(8, "Even array (4x4), odd slice_size=3 -> None", arr2, 3, None)

        # slice_size larger than array -> None
        run_individual_test(9, "slice_size larger than array (5x5, slice=6) -> None", arr, 6, None)

        # Non-square array -> None
        rect = np.arange(12).reshape(3, 4)
        run_individual_test(10, "Non-square array (3x4) -> None", rect, 2, None)

        # Full central region (same size) should return whole array
        arr3 = np.arange(36).reshape(6, 6)
        expected_6x6 = arr3[0:6, 0:6]
        run_individual_test(11, "Slice equal to array size (6x6, slice=6)", arr3, 6, expected_6x6)

        # Larger even example
        arr4 = np.arange(64).reshape(8, 8)
        expected_4x4 = arr4[2:6, 2:6]
        run_individual_test(12, "Central 4x4 from 8x8", arr4, 4, expected_4x4)

        # Larger odd example
        arr5 = np.arange(81).reshape(9, 9)
        expected_5x5 = arr5[2:7, 2:7]
        run_individual_test(13, "Central 5x5 from 9x9", arr5, 5, expected_5x5)

        # 1x1 array, slice=1 valid
        one = np.array([[42]])
        run_individual_test(14, "1x1 array, slice=1 returns the single element", one, 1, one)

        # 2x2 array, slice=1 -> None (even array + odd slice)
        two = np.array([[1, 2], [3, 4]])
        run_individual_test(15, "2x2 array, slice=1 -> None (even/odd mismatch)", two, 1, None)

        # Zero slice -> consider invalid (your function returns None only for certain cases).
        # If you want to treat slice_size=0 as invalid, assert None.
        run_individual_test(16, "slice_size=0 should be invalid -> None (if enforced)", arr2, 0, None)

        # Negative slice_size -> invalid -> None
        run_individual_test(17, "Negative slice_size -> None", arr2, -2, None)

        # Float array type still works (shape-only logic)
        float_arr = np.arange(25, dtype=float).reshape(5, 5)
        expected_float_3x3 = float_arr[1:4, 1:4]
        run_individual_test(18, "Float array central slice (5x5, slice=3)", float_arr, 3, expected_float_3x3)

        # Non-2D array -> should be invalid; your code will break at unpacking shape
        # We defensively expect an exception or None behavior
        try:
            bad = np.arange(8)  # 1D
            res = slice_array(bad, 1)  # type: ignore
            print(f"{RED}Test 19 Warning: 1D input should be rejected, got {res}{RESET}")
        except Exception:
            print(f"{GREEN}Test 19 Passed: 1D input raises as expected{RESET}")

        # 3D array -> invalid
        try:
            bad3d = np.arange(8).reshape(2,2,2)
            res = slice_array(bad3d, 1)  # type: ignore
            print(f"{RED}Test 20 Warning: 3D input should be rejected, got {res}{RESET}")
        except Exception:
            print(f"{GREEN}Test 20 Passed: 3D input raises as expected{RESET}")

        # Non-numeric types (should be fine since slicing ignores dtype)
        obj_arr = np.empty((5,5), dtype=object)
        for i in range(5):
            for j in range(5):
                obj_arr[i, j] = (i, j)
        expected_obj = obj_arr[1:4, 1:4]
        run_individual_test(21, "Object dtype array central slice (5x5, slice=3)", obj_arr, 3, expected_obj)

    # -----------------------------
    # column_sum(arr)
    # -----------------------------
    def test_column_sum():
        def run_individual_test(test_number, description, arr, expected_output):
            try:
                result = column_sum(arr)
                if result is None:
                    raise NotImplementedError("column_sum function is not implemented")
                assert np.array_equal(result, expected_output), f"Expected {expected_output}, got {result}"
                print(f"{GREEN}Test {test_number} Passed: {description}{RESET}")
            except (NotImplementedError, AssertionError, ValueError, TypeError) as e:
                print(f"{RED}Test {test_number} Failed/Skipped: {description}. {str(e)}{RESET}")

        arr = np.array([[1, 2, 3], [4, 5, 6]])
        expected = np.array([5, 7, 9])
        run_individual_test(5, "Sum of columns for 2x3 array", arr, expected)

        # More cases
        run_individual_test(22, "Single row", np.array([[10, -5, 2]]), np.array([10, -5, 2]))
        run_individual_test(23, "Single column", np.array([[1], [2], [3]]), np.array([6]))
        run_individual_test(24, "Zeros", np.zeros((3,4), dtype=int), np.zeros(4, dtype=int))
        run_individual_test(25, "Floats", np.array([[1.5, 2.5], [3.0, -1.0]]), np.array([4.5, 1.5]))

        # Empty with 0 rows but n columns -> sum should be zeros
        empty_rows = np.empty((0, 3))
        run_individual_test(26, "Empty with 0 rows", empty_rows, np.array([0.0, 0.0, 0.0]))

        # NaNs (NumPy sum propagates NaN)
        arr_nan = np.array([[np.nan, 1.0], [2.0, 3.0]])
        expected_nan = np.array([np.nan, 4.0])
        result = column_sum(arr_nan)
        assert np.isnan(result[0]) and result[1] == 4.0, "NaN propagation test failed"
        print(f"{GREEN}Test 27 Passed: NaN propagation in column_sum{RESET}")

    # -----------------------------
    # normalise(arr)
    # -----------------------------
    def test_normalise():
        def run_individual_test(test_number, description, arr, expected_output):
            try:
                result = normalise(arr)
                if result is None:
                    raise NotImplementedError("normalise function is not implemented")
                assert np.allclose(result, expected_output), f"Expected {expected_output}, got {result}"
                assert result.min() == 0 and result.max() == 1, "Normalised array not in [0, 1]"
                assert np.issubdtype(result.dtype, np.floating), "Normalised dtype should be float"
                print(f"{GREEN}Test {test_number} Passed: {description}{RESET}")
            except (NotImplementedError, AssertionError, ValueError, TypeError) as e:
                print(f"{RED}Test {test_number} Failed/Skipped: {description}. {str(e)}{RESET}")

        arr = np.array([[1, 2], [3, 4]])
        expected = np.array([[0, 1/3], [2/3, 1]])
        run_individual_test(6, "Normalise 2x2 array", arr, expected)

        # Negative values
        arr_neg = np.array([[-5, 0], [5, 10]], dtype=float)
        # min=-5, max=10 -> (x+5)/15
        expected_neg = (arr_neg - arr_neg.min()) / (arr_neg.max() - arr_neg.min())
        run_individual_test(28, "Normalise with negatives and positives", arr_neg, expected_neg)

        # Constant array -> ones
        const = np.full((3, 3), 7)
        expected_const = np.ones((3, 3), dtype=float)
        try:
            result = normalise(const)
            assert np.array_equal(result, expected_const), f"Expected ones for constant array, got {result}"
            print(f"{GREEN}Test 29 Passed: Constant array returns ones{RESET}")
        except AssertionError as e:
            print(f"{RED}Test 29 Failed: {str(e)}{RESET}")

        # Larger array sanity
        big = np.arange(100).reshape(10, 10)
        res_big = normalise(big)
        assert np.isclose(res_big.min(), 0) and np.isclose(res_big.max(), 1), "Large normalise bounds"
        print(f"{GREEN}Test 30 Passed: Large array normalisation bounds{RESET}")

        # Float precision checks with allclose
        arr_float = np.array([[0.1, 0.2], [0.3, 0.4]])
        res_float = normalise(arr_float)
        assert np.isclose(res_float.min(), 0) and np.isclose(res_float.max(), 1), "Float normalise bounds"
        print(f"{GREEN}Test 31 Passed: Float input normalisation{RESET}")

    # Run all
    test_create_array()
    test_slice_array()
    test_column_sum()
    test_normalise()


In [ ]:
def test_energy_analysis_tasks():
    def run_test(test_number, description, test_function):
        try:
            test_function()
            print(f"{GREEN}Test {test_number} Passed: {description}{RESET}")
        except AssertionError as e:
            print(f"{RED}Test {test_number} Failed: {description}. {str(e)}{RESET}")
        except Exception as e:
            print(f"{RED}Test {test_number} Error: {e}{RESET}")

    def test_kwh_window_nmi101():
        assert 'nmi101_filtered' in globals(), "nmi101_filtered is not defined"
        # Filter the original energy_usage_df for NMI101 and required hours
        expected_hours = pd.date_range('2023-01-01 10:00', periods=4, freq='h')
        mask = (
            (energy_usage_df['NMI'] == 'NMI101') &
            (energy_usage_df['Hour'].isin(expected_hours))
        )
        expected_values = energy_usage_df.loc[mask, 'kWh'].tolist()
        assert globals()['nmi101_filtered'] == expected_values, (
            f"Expected {expected_values}, got {globals()['nmi101_filtered']}"
        )

    # Test for merging dataframes
    def test_merge_dataframes():
        assert 'merged_df' in globals(), "merged_df is not defined"
        merged_columns = set(customers_df.columns.to_list() + energy_usage_df.columns.to_list()) - {'NMI'}
        assert all(column in globals()['merged_df'].columns for column in merged_columns), "Merged DataFrame missing columns"

    # Test for total energy usage calculation
    def test_total_energy_usage():
        assert 'total_energy_per_customer' in globals(), "total_energy_per_customer is not defined"
        # Perform a sample calculation to verify
        sample_calculation = globals()['merged_df'].groupby('NMI')['kWh'].sum()
        assert globals()['total_energy_per_customer'].equals(sample_calculation), "total_energy_per_customer calculation is incorrect"

    # Test for highest energy usage customer
    def test_highest_energy_usage_customer():
        assert 'highest_energy_usage_customer_name' in globals(), "highest_energy_usage_customer_name is not defined"
        highest_energy_customer = globals()['total_energy_per_customer'].idxmax()
        assert globals()['highest_energy_usage_customer_name'] == highest_energy_customer, "Incorrect highest energy usage customer"

    # Test for average energy usage per hour
    def test_average_energy_per_hour():
        assert 'average_energy_per_hour' in globals(), "average_energy_per_hour is not defined"
        sample_avg_calculation = globals()['merged_df'].groupby(globals()['merged_df']['Hour'])['kWh'].mean()
        assert globals()['average_energy_per_hour'].equals(sample_avg_calculation), "average_energy_per_hour calculation is incorrect"

    # Test for highest usage hour
    def test_highest_usage_hour():
        assert 'highest_usage_hour' in globals(), "highest_usage_hour is not defined"
        highest_hour = globals()['average_energy_per_hour'].idxmax()
        assert globals()['highest_usage_hour'] == highest_hour, "Incorrect highest usage hour"

    def test_age_energy_correlation():
        assert 'age_energy_correlation' in globals(), "age_energy_correlation is not defined"
        merged_df = globals()['merged_df']
        # Option 1: scalar correlation value (Pandas Series)
        expected_scalar = merged_df['Age'].corr(merged_df['kWh'])
        # Option 2: full correlation matrix (Pandas DataFrame)
        expected_matrix = merged_df[['Age', 'kWh']].corr()
        # Accept either answer
        student_value = globals()['age_energy_correlation']
        is_scalar_correct = (
            isinstance(student_value, (float, int))
            and math.isclose(student_value, expected_scalar, rel_tol=0.05)
        )
        is_matrix_correct = (
            hasattr(student_value, 'loc')
            and math.isclose(student_value.loc['Age', 'kWh'], expected_scalar, rel_tol=0.05)
        )
        assert is_scalar_correct or is_matrix_correct, (
            "Incorrect age-energy correlation value: must be either a scalar correlation or a correlation matrix."
        )
        
    # Running the tests
    
    run_test(1, "Extracting kWh values from 10:00am to 1:00pm for NMI101", test_kwh_window_nmi101)
    run_test(2, "Testing merging of DataFrames", test_merge_dataframes)
    run_test(3, "Testing total energy usage calculation", test_total_energy_usage)
    run_test(4, "Testing highest energy usage customer", test_highest_energy_usage_customer)
    run_test(5, "Testing average energy usage per hour", test_average_energy_per_hour)
    run_test(6, "Testing highest usage hour", test_highest_usage_hour)
    run_test(7, "Testing age-energy correlation", test_age_energy_correlation)
